In [2]:
from datasets import load_dataset

# Load the dataset in streaming mode
ds = load_dataset("tiendung/chai", split="train", streaming=True)

# Filter to only examples with thumbs_up == True
filtered_ds = ds.filter(lambda x: x.get("thumbs_up") is True)

# Preview an example
for example in filtered_ds.take(1):
    print(example)

{'_id': '_bot_5c9283de-131d-4600-9818-11456f7458ca_mk4NsyDQzWSsWjxPYkdBOegmJit2_1689211870273_1689211909', 'conversation': [{'name': 'Barbie Roberts (Super Kind)', 'text': 'PACE!'}, {'name': 'Anonymous user', 'text': 'corn on the cob'}, {'name': 'Barbie Roberts (Super Kind)', 'text': '*laughs* what did you just say?'}], 'languages': ['en'], 'thumbs_up': True, 'feedback': 'corn', 'model_name': 'chaiml-ak-edit-v0_1689207647'}


In [ ]:
from data_utils import reformat_chai_old_format

# # Preview a few examples
# for example in filtered_ds.take(3):
#     formatted = convert_chai_old_format(example)
#     print(formatted)
#     print("=" * 60)

all_examples = []
for example in filtered_ds:
    # text = example
    messages = reformat_chai_old_format(example)
    if messages and messages['messages'] != []:  # skip empty ones
        all_examples.append(messages)

In [7]:
import json

with open("../data/chai_conversations_filtered_39k_old_format.jsonl", "w") as f:
    for item in all_examples:
        f.write(json.dumps(item) + "\n")